<a href="https://colab.research.google.com/github/emdi123/Python_AI/blob/master/LSTM_text_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title LSTM for language translation

In [0]:
#Source: https://towardsdatascience.com/word-level-english-to-marathi-neural-machine-translation-using-seq2seq-encoder-decoder-lstm-model-1a913f2dc4a7


In [294]:
#Read the data
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model


############
#Data import
############
lines=pd.read_table("/content/drive/My Drive/AI_projects/Data/deu.txt", names=['Eng', 'De', 'Junk'])
lines = lines[['Eng', 'De']]



##########
#check data
##########
lines.head(3)
# lines.shape

,Eng,De
0,Go.,Geh.
1,Hi.,Hallo!
2,Hi.,Grüß Gott!


In [295]:
"""
The function to process the text for deep learning
lines: Pandas dataframe

output: Pandas dataframe 
"""

def process_txt(lines):

  #################
  #process all data
  #################
  #lines['Eng'].head(20)

  #lowercase all charachters
  lines.Eng= lines.Eng.apply(lambda x: x.lower())
  lines.De= lines.De.apply(lambda x: x.lower())

  #remove quotation marks

  lines.Eng= lines.Eng.apply(lambda x: re.sub("'", '', x))
  lines.De= lines.De.apply(lambda x: re.sub("'", '', x))

  #set of all punctuation marks
  exclude = set(string.punctuation)

  #remove punctuations from the code

  lines.Eng= lines.Eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
  lines.De= lines.De.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

  #remove all numbers from text

  lines.Eng= lines.Eng.apply(lambda x: re.sub("\d", '', x))
  lines.De= lines.De.apply(lambda x: re.sub("\d", '', x))

  #remove extra spaces
  lines.Eng= lines.Eng.apply(lambda x: x.strip())
  lines.De= lines.De.apply(lambda x: x.strip())

  #removing additional symbols
  lines.Eng=lines.Eng.apply(lambda x: re.sub(" +", " ", x))
  lines.De=lines.De.apply(lambda x: re.sub(" +", " ", x))

  return lines

lines1 = process_txt(lines)

'''

Last step in the data processing

input: a Pandas dataframe

'''

def final_process(lines):
  #Add start and end tokens to the target sequence
  lines.De=lines.De.apply(lambda x: 'START_ ' + x+' _END')

  #shuffle the data
  lines = shuffle(lines)

  return lines

lines2 = final_process (lines)

##########################
#Checking data
##########################

lines.head(20)
len(lines.Eng)

print(lines.tail())

                        Eng                                         De
41864  tom slept restlessly            START_ tom schlief unruhig _END
41865  tom slipped and fell  START_ tom rutschte aus und fiel hin _END
41866  tom smiled nervously            START_ tom lächelte nervös _END
41867  tom sold his company      START_ tom verkaufte seine firma _END
41868  tom sold his company                             START_ to _END


In [296]:
lines.tail(5)

,Eng,De
41864,tom slept restlessly,START_ tom schlief unruhig _END
41865,tom slipped and fell,START_ tom rutschte aus und fiel hin _END
41866,tom smiled nervously,START_ tom lächelte nervös _END
41867,tom sold his company,START_ tom verkaufte seine firma _END
41868,tom sold his company,START_ to _END


In [297]:
'''
This function creates the language vocabulary
input : pandas dataframe thaat is processed for text processing
'''
def create_vocab(lines): 


#############################
# Information retrival for data
############################

  # Vocabulary of English
  # All unique letters in the english words
  # They are in sets because sets cannot contain duplicates

  all_eng_words=set()
  for eng in lines.Eng:
      for word in eng.split():
          if word not in all_eng_words:
              all_eng_words.add(word)

  # Vocabulary of German
  all_de_words=set()
  for de in lines.De:
      for word in de.split():
          if word not in all_de_words:
              all_de_words.add(word)

  return all_eng_words, all_de_words


#####################
#return function
#####################
all_eng_words, all_de_words = create_vocab(lines)


# Max Length of source sequence 
lenght_list=[]
for l in lines.Eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)



# Max Length of target sequence
lenght_list=[]
for l in lines.De:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
print('Max length (charachters) of target sequence is '+ str(max_length_tar))


#Sorting the list
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_de_words))

#Here length is the length of all the words together
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_de_words)

print("Total number of encorders are %s and total number of decorders are %s "% (num_encoder_tokens,num_decoder_tokens ))

num_decoder_tokens += 1 # For zero padding
num_encoder_tokens += 1 # For zero padding

#Assigning a digit to each word
# Convert it into a dictionary
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

#reversing the order
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

#print("The lenght of source sequence " + str(num_encoder_tokens) + '\n' + "The length of target sequence " + str(num_decoder_tokens) )


Max length (charachters) of target sequence is 17
Total number of encorders are 5661 and total number of decorders are 9274 


In [0]:
####################
#Modeling - making ready with tools
####################

X, y= lines.Eng, lines.De
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.1)

#picke the results to use later

X_train.to_pickle("/content/drive/My Drive/AI_projects/Models/X_train_translate1.pkl")
X_test.to_pickle("/content/drive/My Drive/AI_projects/Models/X_test_translate1.pkl")


#####################
###################
#DID NOT UNDERSTAND
######################

def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)





In [300]:
X_train

5248           i married tom
6464           tom will talk
36093    which beer is yours
21172      it was depressing
28519     tom has a tricycle
                ...         
5434            im a cripple
14445       do you play golf
9276          tom needs this
24457      you take this one
16491        please dont die
Name: Eng, Length: 37682, dtype: object

In [301]:
################
#Create the model
#This model converts encorder_input_data and decorder_input_data into 
#decorder_output_data
################


latent_dim = 50
########################
#Setting up the encorder
encorder_inputs = Input(shape= (None,))

#Here we embedd the input
#output dimension is fixed at latent_dim
#Here we provide the length of the sequence.
enc_emb= Embedding(num_encoder_tokens,latent_dim, mask_zero= True )(encorder_inputs)
encoder_lstm = LSTM(latent_dim, return_state= True)
encorder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encorder_states = [state_h, state_c] # only the states are taken

######################
#setting up the decorder

decorder_inputs = Input(shape= (None,))

#Here we embedd the input
#output dimension is fixed at latent_dim
dec_emb_layer= Embedding(num_decoder_tokens,latent_dim, mask_zero= True )

#################
################
#DIDNOT UMDERSTNAD THIS STEP
dec_emb = dec_emb_layer(decorder_inputs)
#LSTM function call 
decoder_lstm = LSTM(latent_dim, return_state= True,return_sequences= True)

#DID NOT UNDERSTAND THIS
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encorder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encorder_inputs, decorder_inputs], decoder_outputs)

##################
#compile the model
##################

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

#plot the results

plot_model(model, show_shapes=True, to_file= "/content/drive/My Drive/AI_projects/Figures/lstm_translator.png")
model.summary()

Model: "model_35"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_49 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_50 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_26 (Embedding)        (None, None, 50)     283100      input_49[0][0]                   
__________________________________________________________________________________________________
embedding_27 (Embedding)        (None, None, 50)     463750      input_50[0][0]                   
___________________________________________________________________________________________

In [0]:
##########################
#MOdel fitting
##########################
train_samples = len(X_train) # here training sample size will be bigger than the vocabilary size
val_samples = len(X_test)
batch_size = 128
epochs = 50

#######################
#Run only if needed 
######################

#model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    # steps_per_epoch = train_samples//batch_size,
                    # epochs=epochs,
                    # validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    # validation_steps = val_samples//batch_size)
#save the weights
#model.save_weights("/content/drive/My Drive/AI_projects/Models/nmt_weights.h5")


In [0]:

#load the weights if needed
model.load_weights("/content/drive/My Drive/AI_projects/Models/nmt_weights.h5")

In [0]:
##########################
#Setting up the interference
##########################
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encorder_inputs, encorder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape= (latent_dim,))
decoder_states_inputs = [decoder_state_input_h,decoder_state_input_c]

#give the new inputs to the decorder layer created above
dec_emb2= dec_emb_layer(decorder_inputs)

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state= decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2)

#Final decorder model
# Final decoder model
decoder_model = Model([decorder_inputs] + decoder_states_inputs,
                      [decoder_outputs2] + decoder_states2
                      )




In [0]:
###########################
#Setting up the decorder
###########################
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [308]:
#####################
#Evaluation the results
####################

train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1


k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual German Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted German Translation:', decoded_sentence[:-4])


X_train[k:k+1].values[0]


Input English sentence: i married tom
Actual German Translation:  ich heiratete tom 
Predicted German Translation:  ich habe tom gesagt 


'i married tom'

In [0]:
#@title SECOND EXAMPLE

#Souurce: https://nextjournal.com/gkoehler/machine-translation-seq2seq-cpu


In [1]:
############
#Data import
############
with open("/content/drive/My Drive/AI_projects/Data/deu.txt", 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')

print(len(lines))

#splitting into input and target
input_texts = []
target_texts = []

input_characters = set()
target_characters = set()


num_samples = 10000

#go through line by line, fill in the input sequence , target sequence, input and target  charachters 
for line in lines[: min(num_samples, len(lines) - 1)]:
  input_text, target_text,junk  = line.split('\t')
  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)


#let's take a sample
print(input_texts[155], target_texts[155])
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

print('##############')
print(input_characters)

#tokenize our charachters
input_token_index = dict(
  [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
  [(char, i) for i, char in enumerate(target_characters)])


#creating numeric data
import numpy as np

encoder_input_data = np.zeros(
  (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
  dtype='float32')
decoder_input_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')
decoder_target_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')

print("\n Shape of encorder input is " + str(encoder_input_data.shape))

print("\n Shape of decorder input is " + str(decoder_target_data.shape))

#filling in ith actual tokens
#we have to offset decoder_target_data by one timestep.

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.
  for t, char in enumerate(target_text):
    # decoder_target_data is ahead of decoder_input_data by one timestep
    decoder_input_data[i, t, target_token_index[char]] = 1.
    if t > 0:
      # decoder_target_data will be ahead by one timestep
      # and will not include the start character.
      decoder_target_data[i, t - 1, target_token_index[char]] = 1.




41869
Go home. 	Geh heim.

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 87
Max sequence length for inputs: 15
Max sequence length for outputs: 45
##############
[' ', '!', '"', '$', '%', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

 Shape of encorder input is (10000, 15, 71)

 Shape of decorder input is (10000, 45, 87)


In [3]:
################
#Building the model
################
import keras, tensorflow
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

#hyperparameters
#dimensionality for the encoder and decoder LSTM layers have to match

#batch_size is the number of samples taken for training step.
#For instance, here the network takes 64 samples for the first batch of training
batch_size = 64  # batch size for training

#one epoch is when the entire dataset is passed one time forward and backard 
epochs = 100  # number of epochs to train for

#
latent_dim = 256  # latent dimensionality of the encoding space

#model

#num_encoder_tokens is the number of unique input characters
#shape = (,x) : here x is the dimentionnality of the input, None indicates that batch size is arbitery
encoder_inputs = Input(shape=(None, num_encoder_tokens))


encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]


#model architecture
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#Use the keras API
model = Model(inputs=[encoder_inputs, decoder_inputs], 
              outputs=decoder_outputs)


model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()



Using TensorFlow backend.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 71)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 87)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 335872      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  352256      input_2[0][0]                    
                                                                 lstm_1[0][1]               

In [22]:
help(LSTM)

Help on class LSTM in module keras.layers.recurrent:

class LSTM(RNN)
 |  Long Short-Term Memory layer - Hochreiter 1997.
 |  
 |  # Arguments
 |      units: Positive integer, dimensionality of the output space.
 |      activation: Activation function to use
 |          (see [activations](../activations.md)).
 |          Default: hyperbolic tangent (`tanh`).
 |          If you pass `None`, no activation is applied
 |          (ie. "linear" activation: `a(x) = x`).
 |      recurrent_activation: Activation function to use
 |          for the recurrent step
 |          (see [activations](../activations.md)).
 |          Default: sigmoid (`sigmoid`).
 |          If you pass `None`, no activation is applied
 |          (ie. "linear" activation: `a(x) = x`).
 |      use_bias: Boolean, whether the layer uses a bias vector.
 |      kernel_initializer: Initializer for the `kernel` weights matrix,
 |          used for the linear transformation of the inputs.
 |          (see [initializers](../in

In [4]:
#Model fitting
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
model.save_weights("/content/drive/My Drive/AI_projects/Models/nmt_weights_secondModel.h5")

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 23s 3ms/step - loss: 1.1720 - val_loss: 1.1310
Epoch 2/100
8000/8000 [==============================] - 21s 3ms/step - loss: 0.9125 - val_loss: 0.9110
Epoch 3/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.7661 - val_loss: 0.8114
Epoch 4/100
8000/8000 [==============================] - 21s 3ms/step - loss: 0.6911 - val_loss: 0.7404
Epoch 5/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.6428 - val_loss: 0.7041
Epoch 6/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.6050 - val_loss: 0.6732
Epoch 7/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.5743 - val_loss: 0.6491
Epoch 8/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.5478 - val_loss: 0.6313
Epoch 9/100
8000/8000 [==============================] - 20s 3ms/step - loss: 0.5245 - val_loss: 0.6069
Epoch 10/100
800

In [0]:
#load the weights
model.load_weights("/content/drive/My Drive/AI_projects/Models/nmt_weights_secondModel.h5")



In [0]:
#Interface
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
  decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
  [decoder_inputs] + decoder_states_inputs,
  [decoder_outputs] + decoder_states)

# reverse-lookup token index to turn sequences back to characters
reverse_input_char_index = dict(
  (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
  (i, char) for char, i in target_token_index.items())




In [0]:
#Decoding the sequence
def decode_sequence(input_seq):
  # encode the input sequence to get the internal state vectors.
  states_value = encoder_model.predict(input_seq)
  
  # generate empty target sequence of length 1 with only the start character
  target_seq = np.zeros((1, 1, num_decoder_tokens))
  target_seq[0, 0, target_token_index['\t']] = 1.
  
  # output sequence loop
  stop_condition = False
  decoded_sentence = ''
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict(
      [target_seq] + states_value)
    
    # sample a token and add the corresponding character to the 
    # decoded sequence
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = reverse_target_char_index[sampled_token_index]
    decoded_sentence += sampled_char
    
    # check for the exit condition: either hitting max length
    # or predicting the 'stop' character
    if (sampled_char == '\n' or 
        len(decoded_sentence) > max_decoder_seq_length):
      stop_condition = True
      
    # update the target sequence (length 1).
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.
    
    # update states
    states_value = [h, c]
    
  return decoded_sentence

In [19]:
#test the model 
input_sentence = " do you have?"
test_sentence_tokenized = np.zeros(
  (1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
for t, char in enumerate(input_sentence):
  test_sentence_tokenized[0, t, input_token_index[char]] = 1.
print(input_sentence)
print(decode_sequence(test_sentence_tokenized))

 do you have ?
Ich wurde betrennen.

